<a href="https://colab.research.google.com/github/duyvm/leetcode-problems/blob/main/%5BMED%5D3362_Zero_Array_Transformation_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3362. Zero Array Transformation III

https://leetcode.com/problems/zero-array-transformation-iii/description/

You are given an integer array `nums` of length `n` and a 2D array `queries` where `queries[i] = [l`<sub>`i`</sub>`, r`<sub>`i`</sub>`]`.

Each `queries[i]` represents the following action on nums:

- Decrement the value at each index in the range `[l`<sub>`i`</sub>`, r`<sub>`i`</sub>`]` in `nums` by at most `1`.

- The amount by which the value is decremented can be chosen independently for each index.

A **Zero Array** is an array with all its elements equal to `0`.

Return the **maximum number of elements** that can be removed from queries, such that nums can still be converted to a zero array using the remaining queries. If it is not possible to convert `nums` to a zero array, return `-1`.

**Constraints:**
- `1 <= nums.length <= 10`<sup>`5`</sup>
- `0 <= nums[i] <= 10`<sup>`5`</sup>
- `1 <= queries.length <= 10`<sup>`5`</sup>
- `queries[i].length == 2`
- `0 <= l`<sub>`i`</sub>` <= r`<sub>`i`</sub>` < nums.length`

**Example 1**

After removing `queries[2]`, `nums` can still be converted to a zero array.

- Using `queries[0]`, decrement `nums[0]` and `nums[2]` by `1` and `nums[1]` by `0`.
- Using `queries[1]`, decrement `nums[0]` and `nums[2]` by `1` and `nums[1]` by `0`.

In [4]:
test_case1 =  {
            "input": {
                "nums": [2,0,2],
                "queries": [[0,2],[0,2],[1,1]],
                "isReturnResult": True,
            },
            "output": 1
        }

**Example 2**

We can remove `queries[2]` and `queries[3]`.

In [5]:
test_case2 =  {
            "input": {
                "nums": [1,1,1,1],
                "queries": [[1,3],[0,2],[1,3],[1,2]],
                "isReturnResult": True,
            },
            "output": 2
        }

**Example 3**

`nums` cannot be converted to a zero array even after using all the queries.

In [6]:
test_case3 =  {
            "input": {
                "nums": [1,2,3,4],
                "queries": [[0,3]],
                "isReturnResult": True,
            },
            "output": -1
        }

test_case4 =  {
            "input": {
                "nums": [0,3],
                "queries": [[0,1],[0,0],[0,1],[0,1],[0,0]],
                "isReturnResult": True,
            },
            "output": 2
        }

test_case5 =  {
            "input": {
                "nums": [0,0,1,1,0],
                "queries": [[3,4],[0,2],[2,3]],
                "isReturnResult": True,
            },
            "output": 2
        }

In [7]:
test_cases = [test_case1, test_case2, test_case3, test_case4, test_case5]

In [8]:
def run_test_cases(solution, function_name):
    for i in range(len(test_cases)):
        run_test_case(solution, function_name, i)

def run_test_case(solution, function_name, i):
    test_case = test_cases[i]
    result = getattr(solution, function_name)(**test_case["input"])
    if result != test_case["output"]:
        print(f'Failed test case {i} with input: {test_case["input"]} and expected result: {test_case["output"]} vs actual result: {result}')

# Approach 1

## Observations

- Credit go to [here](https://leetcode.com/problems/zero-array-transformation-iii/solutions/6768535/priority-queues-in-depth-with-images-idea-behind-solution-c-python-java)

- Let `m` is `queries.length`

- New version of [Problem 3356. Zero Array Transformation II](https://colab.research.google.com/drive/1wCVV_v5q1_v9uPtkSP2mnuEm2hfp_0Ot). Differences:

  - The decrement amount is `0` or `1`.

  - The required answer is slightly different. In **3356**, return minimum number of first `k` queries that make `nums` **Zero Array** while this problem's required answer is maxinum number of un-needed queries. In context of **3356**, it is the counterpart: `m - k` number of queries.

  - Furthermore, in this problem, the un-needed queries is not necsessary continuous (aka, not the last `m-k` queries in `queries` in case of **3356**).

- We must selectively choose the query that make `nums` **Zero Array** faster.

  - Since the decrease amount is fixed to `0` or `1`, idealy, we should choose the query that maximizes the impact the decrement, a.k.a `l - r` is maximum.

  - The executing order of queries is not matter, we will consider start decrement `nums` from left to right. We pick up query that has smallest `l` first. To do this, we sort `queries` by `query[0]` of each `query`

  - Let `nums[i]` is the current element for `i` from `0` to `n-1`.
  
  - Let `considered_queries` is a heap that store all the queries that we consider them may affect `nums[i]`.
  
    - Starting from `queries[0]`, we add query that has `l <= i`
  into `considered_queries`, having query that have largest `r` in the top of this heap.
  
    - In this way, when we pick query from `considered_queries`, we can get query that has largest affect on `nums`.

  - We use another heap `used_queries` to keep track of the queries we actually are using for current `nums[i]`.
     
     - At iteration round `i`, we check if the top query is still affect `nums[i]` (`r >= i`), if not, we remove it until all queries left in `used_queries` are affecting `nums[i]`. The number of queries in `used_queries` is equavilent to the decrement value at `nums[i]`.
     
     - If `used_queries.length < nums[i]`, that mean we still need more queries to make `nums[i] = 0`. So, we must get new query from `considered_queries` that has `r >= i` and add it to `used_queries`.

     - If `used_queries.length >= nums[i]` then `nums[i]` can be `0` with current `used_queries`, we proceed to `i+1` without getting new query from `considered_queries`

    - If after getting all possible queries from `considered_queries` but can not make `used_queries.length >= nums[i]`, then it means that there are not enough queries that can make `nums[i]` to `0`. We consider it is impossible to convert `nums` to a zero array and return `-1`

- While adding new query into `used_queries`, we use `count` to keep track of total number of queries we used. We return `queries.length - count`, which is the number of un-used queries, after finish iteration.

- Furthermore, we do not need to store whole query `[l, r]` inside both heaps but only the `r` value is enough.

## Analysis

- Time complexity:

  - One pass through `nums`: `O(n)`

  - Sorting the `queries`: `O(mlog(m))`

  - One pass through `queries` using `k`: `O(m)`

  - Insertion/deletion from heap with worth case (heap has full queries): `O(log(m))`

  - Total time: `O(n + mlog(m))`

### Implementation

In [21]:
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:
    def maxRemoval(self, nums: List[int], queries: List[List[int]], isReturnResult=False) -> int:

        n, m = len(nums), len(queries)

        queries = sorted(queries, key=lambda query: query[0])

        considered_queries = []

        used_queries = []

        k = 0

        count = 0

        for i in range(n):
            while used_queries and used_queries[0] < i:
                heappop(used_queries)

            while k < m and queries[k][0] <= i:
                heappush(considered_queries, -queries[k][1])
                k += 1

            while len(used_queries) < nums[i] and considered_queries and -considered_queries[0] >= i:
                heappush(used_queries, -heappop(considered_queries))
                count += 1

            if len(used_queries) < nums[i]:
                return -1

        return m - count

In [22]:
run_test_cases(Solution(), "maxRemoval")